In [1]:
import numpy as np
import h5py
import torch 
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms

In [2]:
with h5py.File('dataset/Electron.hdf5','r') as h5:
    ElectronX = h5['X'][:]
    ElectronY = h5['y'][:]

with h5py.File('Photon.hdf5','r') as h5:
    PhotonX = h5['X'][:]
    PhotonY = h5['y'][:]

ElectronX.shape, PhotonX.shape, ElectronY.shape, PhotonY.shape

((249000, 32, 32, 2), (249000, 32, 32, 2), (249000,), (249000,))

In [3]:
X = np.concatenate([ElectronX, PhotonX])
Y = np.concatenate([ElectronY, PhotonY])

indexes = np.random.permutation(len(X))
X = X[indexes]
Y = Y[indexes]

x_train, x_test = np.split(X, [int(0.8*len(X))])
y_train, y_test = np.split(Y, [int(0.8*len(Y))])

In [4]:
np.sum(y_train == 0) / len(y_train), np.sum(y_test == 0) / len(y_test) 

(0.5004869477911646, 0.49805220883534135)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, transform=None, label_transform = None):
        self.X = X
        self.labels = Y
        self.transform = transform
        self.label_transform = label_transform


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = self.X[idx]
        label = self.labels[idx]
        
        if self.transform:
            sample = (self.transform(sample))

        if self.label_transform:
            label = np.array(label)
            label = self.label_transform(label)

        return sample, label

CustomTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5], [0.5, 0.5])
])

def OneHotEncoder(label):
    return torch.nn.functional.one_hot(label, 2)

trainDS = CustomDataset(x_train, y_train, CustomTransform)
testDS = CustomDataset(x_test, y_test, CustomTransform)

trainDL = DataLoader(trainDS, batch_size=32, shuffle=True)
testDL =  DataLoader(testDS,  batch_size=32, shuffle=True)

In [6]:
class ConvLayerStack(nn.Module):

    def __init__(self, in_c, out_c, kernel, stride, padding):
        super().__init__()
        self.Conv1 = nn.Conv2d(in_c, in_c//4, kernel, stride, padding)
        # self.Conv2 = nn.Conv2D(in_c/4, in_c/4, kernel, stride, padding)
        self.Conv3 = nn.Conv2d(in_c//4, out_c, kernel, stride, padding)
        self.BNorm1 = nn.BatchNorm2d(in_c//4)
        self.BNorm2 = nn.BatchNorm2d(out_c)
        self.Relu = nn.ReLU()

    def forward(self, x):
        f = self.Conv3( self.Relu( self.BNorm1(self.Conv1(x)) ))
        # print(f.shape)
        return self.Relu(self.BNorm2(f + x))


class ResNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.Conv1 = nn.Conv2d(2,64,2,2,0)
        self.Pooling1 = nn.AvgPool2d(1,1)
        self.Res1 = ConvLayerStack(64,64,3,1,1)
        self.Res2 = ConvLayerStack(64,64,3,1,1)
        self.Res3 = ConvLayerStack(64,64,3,1,1)
        self.Res4 = ConvLayerStack(64,64,3,1,1)
        self.Res5 = ConvLayerStack(64,64,3,1,1)
        self.Res6 = ConvLayerStack(64,64,3,1,1)
        self.Pooling2 = nn.AvgPool2d(2,2,0)
        self.Lin = nn.Linear(64*8*8, 2)

    def forward(self,x):
        # print(x.shape)
        f = self.Conv1(x)
        # print(f.shape)
        f = self.Pooling1(f)
        # print(f.shape)
        f = self.Res1(f)
        f = self.Res2(f)
        f = self.Res3(f)
        f = self.Res4(f)
        f = self.Res5(f)
        f = self.Res6(f)
        # print(f.shape)
        f = self.Pooling2(f)
        # print(f.shape)
        f = torch.flatten(f,1)
        # print(f.shape)
        return self.Lin(f)
        

In [7]:
from tqdm import tqdm
import torch
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

def Train(model, data, opti, loss_fn):
    model.train()
    Dsize = len(data.dataset)
    NBatches = len(data)

    loss = 0.
    correct = 0
    
    for (x,y) in tqdm(data, total=NBatches):
        x, y = x.to("cuda"), y.to("cuda")
        pred = model(x)
        loss_current = loss_fn(pred, y.long())

        opti.zero_grad()
        loss_current.backward()
        opti.step()
        opti.zero_grad()

        loss += loss_current.item()
        correct += (pred.argmax(1) == y.long()).type(torch.float).sum().item()

    print(f"Training Error: {1 - (correct/Dsize)}, Loss: {loss/NBatches}")

def Test(model, dataloader, criterion, device='cuda'):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)  # shape: (batch_size, 2)
            loss = criterion(outputs, labels.long())
            
            # Accumulate loss
            total_loss += loss.item() * inputs.size(0)
            
            # Predictions and accuracy
            preds = torch.argmax(outputs, dim=1)  # Get predicted class
            total_correct += (preds == labels.long()).sum().item()
            total_samples += labels.size(0)
            
            # Compute probabilities for AUC
            probs = F.softmax(outputs, dim=1)[:, 1]  # Probability of class 1
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute average loss
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    # Compute AUC score
    auc_score = roc_auc_score(all_labels, all_probs)

    print(f"Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}")
    return avg_loss, accuracy, auc_score
        

In [8]:
ResNN = ResNet().to("cuda")

loss_fn = nn.CrossEntropyLoss()
opti = torch.optim.RMSprop(ResNN.parameters(), lr=10e-5)

In [9]:
print(ResNN)

ResNet(
  (Conv1): Conv2d(2, 64, kernel_size=(2, 2), stride=(2, 2))
  (Pooling1): AvgPool2d(kernel_size=1, stride=1, padding=0)
  (Res1): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Conv3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu): ReLU()
  )
  (Res2): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Conv3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Relu): ReLU()
  )
  (Res3): ConvLayerStack(
    (Conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(

In [10]:
epochs = 15
max_acc = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [01:56<00:00, 106.51it/s]


Training Error: 0.37219377510040164, Loss: 0.6457581309381738
Loss: 0.6432, Accuracy: 0.6408, AUC: 0.7087
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [01:58<00:00, 105.30it/s]


Training Error: 0.32502259036144576, Loss: 0.60690364704314
Loss: 0.9076, Accuracy: 0.5025, AUC: 0.4361
Epoch 3
-------------------------------


...

Training Error: 0.2750778112449799, Loss: 0.5506373483517084
Loss: 0.6729, Accuracy: 0.6505, AUC: 0.7522
Epoch 14
-------------------------------


100%|██████████| 12450/12450 [01:56<00:00, 106.99it/s]


Training Error: 0.27329568273092375, Loss: 0.5484840680413456
Loss: 0.6502, Accuracy: 0.6702, AUC: 0.7667
Epoch 15
-------------------------------


100%|██████████| 12450/12450 [01:56<00:00, 106.77it/s]


Training Error: 0.27080572289156624, Loss: 0.5461049442382223
Loss: 0.5734, Accuracy: 0.7069, AUC: 0.7788


In [11]:
max_acc

0.7092771084337349

In [12]:
opti = torch.optim.RMSprop(ResNN.parameters(), lr=10e-8)

In [13]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [01:56<00:00, 106.54it/s]


Training Error: 0.2644126506024096, Loss: 0.535971604171527
Loss: 0.5567, Accuracy: 0.7206, AUC: 0.7866
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [01:56<00:00, 106.63it/s]


Training Error: 0.2632856425702811, Loss: 0.5347254251691711
Loss: 0.5559, Accuracy: 0.7210, AUC: 0.7873
Epoch 3
-------------------------------


...

Training Error: 0.2603187751004016, Loss: 0.5302947604057779
Loss: 0.5546, Accuracy: 0.7230, AUC: 0.7891
Epoch 14
-------------------------------


100%|██████████| 12450/12450 [01:59<00:00, 103.88it/s]


Training Error: 0.259894578313253, Loss: 0.5301153304897638
Loss: 0.5542, Accuracy: 0.7229, AUC: 0.7891
Epoch 15
-------------------------------


100%|██████████| 12450/12450 [02:00<00:00, 103.51it/s]


Training Error: 0.26010040160642567, Loss: 0.5301257873132047
Loss: 0.5544, Accuracy: 0.7231, AUC: 0.7891


In [14]:
max_acc

0.7230722891566265

# Final result:
We obtain:
Best *Validation* accuracy: *0.7238*<br>
Best *Traning* accuracy: *0.7439*<br>
Best *AUC* Score is: *0.7887*<br>


In [16]:
opti = torch.optim.Adam(ResNN.parameters(), lr=10e-9)

In [ ]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    Train(ResNN, trainDL, opti, loss_fn,)
    a, cur_acc, b = Test(ResNN, testDL, loss_fn)
    if cur_acc >= max_acc:
        filename = "BestModel.pth"
        torch.save(ResNN, filename)
        max_acc = cur_acc

Epoch 1
-------------------------------


100%|██████████| 12450/12450 [02:02<00:00, 101.46it/s]


Training Error: 0.2600225903614458, Loss: 0.5298766680462772
Loss: 0.5542, Accuracy: 0.7230, AUC: 0.7892
Epoch 2
-------------------------------


100%|██████████| 12450/12450 [02:01<00:00, 102.26it/s]


Training Error: 0.2599748995983936, Loss: 0.5297630134667738
Loss: 0.5543, Accuracy: 0.7228, AUC: 0.7892
Epoch 3
-------------------------------


...

100%|██████████| 12450/12450 [02:00<00:00, 103.11it/s]


Training Error: 0.2598694779116466, Loss: 0.5297961108224938
Loss: 0.5543, Accuracy: 0.7231, AUC: 0.7892
Epoch 7
-------------------------------


100%|██████████| 12450/12450 [02:00<00:00, 103.50it/s]


Training Error: 0.25962098393574295, Loss: 0.5296393323471269
Loss: 0.5542, Accuracy: 0.7228, AUC: 0.7893
Epoch 8
-------------------------------


 69%|██████▉   | 8583/12450 [01:20<00:35, 107.42it/s]